In [1]:
import cPickle
from collections import OrderedDict
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import gp_emulator
from eoldas_ng import *
from tip_helpers import StandardStateTIP, ObservationOperatorTIP

/opt/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
state_config = OrderedDict()
state_config['omega_vis'] = VARIABLE
state_config['d_vis'] = VARIABLE
state_config['a_vis'] = VARIABLE
state_config['omega_nir'] = VARIABLE
state_config['d_nir'] = VARIABLE
state_config['a_nir'] = VARIABLE
state_config['lai'] = VARIABLE

state_grid = np.arange(1, 366)

the_state = StandardStateTIP ( state_config, state_grid )



Saving results to eoldas_retval_20160721_141222_tyche


In [3]:
gp_vis = cPickle.load(open("tip_vis_albedo_transformed.pkl", 'r'))
gp_nir = cPickle.load(open("tip_nir_albedo_transformed.pkl", 'r'))

obs = np.loadtxt("synthetic_albedo.txt")
mask = np.c_[ obs[:,0], np.ones(obs.shape[0]) ]
observations = obs[:,1:]
print observations.shape, mask.shape


(23, 2) (23, 2)


In [4]:
obsop = ObservationOperatorTIP ( state_grid, the_state, observations, mask, [gp_vis, gp_nir], np.array([0.001, 0.001]) )

In [5]:
obsop.time_step(5)

array([ 0.0915,  0.1844])

In [6]:
x_dict = OrderedDict()
x_dict['omega_vis'] = np.ones_like ( state_grid )*0.17
x_dict['d_vis'] = np.exp(-1.*np.ones_like ( state_grid))
x_dict['a_vis'] = np.ones_like ( state_grid)*0.1
x_dict['omega_nir'] = np.ones_like ( state_grid )*0.7
x_dict['d_nir'] = np.exp(-2.*np.ones_like ( state_grid))
x_dict['a_nir'] = np.ones_like ( state_grid)*0.18
x_dict['lai'] = np.exp(-1.5*2.)
obsop.der_cost ( x_dict, state_config )

AssertionError: 

In [ ]:
%debug

> /home/ucfajlg/.local/lib/python2.7/site-packages/gp_emulator/GaussianProcess.py(235)predict()
    233 
    234         ( nn, D ) = testing.shape
--> 235         assert D == self.D
    236 
    237         expX = np.exp ( self.theta )

ipdb> up
> /data/netapp_3/ucfajlg/python/twostream/tip_helpers.py(241)der_cost()
    239         # uses the GP output from here
    240         self.fwd_albedo_vis, self.dfwd_albedo_vis= self.emulators[0].predict (
--> 241                             x_params[[0,1,2,6], :] )
    242         self.fwd_albedo_nir, self.dfwd_albedo_nir = self.emulators[0].predict ( 
    243                             x_params[[3,4,5,6], :] )

ipdb> print x_params.shape
(7, 365)
